In [1]:
library(tidyverse)
library(gifski)
library(gganimate)
library(plotly)
library(RColorBrewer)
library(wesanderson)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: 'plotly'


The following object is masked from 'package:ggplot2':

    last_plot


The following object is masked from 'package:stats':

    filter


The following object is masked from 'package:graphics':

    layout




In [2]:
# FUNCIONES
normalizarfecha <- function(fecha){
return(str_replace_all(fecha,"X","")  %>% str_replace_all("\\.","/"))
}

cod_Meses <- function(x) {
    labels_meses <- c(
    "Ene","Feb","Mar","Abr","May","Jun",
    "Jul","Ago","Sep","Oct","Nov","Dic")
    return(factor(x,levels = (1:12),labels = labels_meses))
}


cod_dias <- function(x) {
    labels_dias <- c("Miercoles","Jueves","Viernes","Sabado","Domingo","Lunes","Martes")
    return(factor(x,levels = (1:7),labels = labels_dias))
}


In [15]:
# Codigo limpieza y manipulación 
casos_diarios <-  read.csv(file = "Casos_Diarios_Estado_Nacional.csv")
casos_diario_list <- gather(casos_diarios,key ="dias",value="infectados",4:856)

fecha = map(casos_diario_list[4],normalizarfecha)
casos_diarios_fecha <- cbind(casos_diario_list[c(TRUE,TRUE,TRUE,FALSE,TRUE)],fecha)
casos_diarios_fecha <- casos_diarios_fecha[c(1,2,3,5,4)]

datos <- separate( data = casos_diarios_fecha,
                   col =dias,
                   into = c("dia","mes","año"),
                   sep = '/',
                   remove = FALSE)
head(datos)

,cve_ent,poblacion,nombre,dias,dia,mes,año,infectados
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,1,1434635,AGUASCALIENTES,26/02/2020,26,02,2020,0
2,2,3634868,BAJA CALIFORNIA,26/02/2020,26,02,2020,0
3,3,804708,BAJA CALIFORNIA SUR,26/02/2020,26,02,2020,0
4,4,1000617,CAMPECHE,26/02/2020,26,02,2020,0
5,7,5730367,CHIAPAS,26/02/2020,26,02,2020,0
6,8,3801487,CHIHUAHUA,26/02/2020,26,02,2020,0


In [16]:
# usamos la funcion cod_Meses para convertir la columna mes de numerica a string
# Añadimos una variable llamada consecutivo con longitud lenght(datos)

datos <- datos %>% 
    mutate(mest_txt = cod_Meses(as.integer(mes))) %>% 
    mutate(consecutivo = 1:28149) %>% 
    transform(dias = as.Date(dias,"%d/%m/%Y"))
head(datos)

#Filtramos la BD para observar unicamente los datos Nacionales ()
datosnacional <-  datos %>% 
    filter(nombre == "Nacional") %>% 
    mutate(consecutivo = 1:853)

head(datosnacional)



,cve_ent,poblacion,nombre,dias,dia,mes,año,infectados,mest_txt,consecutivo
,<int>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<int>,<fct>,<int>
1,1,1434635,AGUASCALIENTES,2020-02-26,26,02,2020,0,Feb,1
2,2,3634868,BAJA CALIFORNIA,2020-02-26,26,02,2020,0,Feb,2
3,3,804708,BAJA CALIFORNIA SUR,2020-02-26,26,02,2020,0,Feb,3
4,4,1000617,CAMPECHE,2020-02-26,26,02,2020,0,Feb,4
5,7,5730367,CHIAPAS,2020-02-26,26,02,2020,0,Feb,5
6,8,3801487,CHIHUAHUA,2020-02-26,26,02,2020,0,Feb,6


,cve_ent,poblacion,nombre,dias,dia,mes,año,infectados,mest_txt,consecutivo
,<int>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<int>,<fct>,<int>
1,0,127792286,Nacional,2020-02-26,26,02,2020,0,Feb,1
2,0,127792286,Nacional,2020-02-27,27,02,2020,4,Feb,2
3,0,127792286,Nacional,2020-02-28,28,02,2020,2,Feb,3
4,0,127792286,Nacional,2020-02-29,29,02,2020,2,Feb,4
5,0,127792286,Nacional,2020-03-01,01,03,2020,2,Mar,5
6,0,127792286,Nacional,2020-03-02,02,03,2020,5,Mar,6


In [22]:
# Nuevamente usamos un vector consecutivo y uno ciclico para codificar los dias de la semana de los datos Nacionales
consecutivo <- 1:853 
dia_sem <- 1:7 

data_com <- cbind(consecutivo,dia_sem) 

# Unimos los dos objetos anteriores data_com y datosnacional en formato largo
NacionalDat <- merge(datosnacional,as.data.frame(data_com),by = c("consecutivo")) #formato largo

# Usamos la funcion cod_dias para codificar la variable de numerica a string
NacionalDat <- NacionalDat %>%   
    mutate(dias_txt = cod_dias(as.integer(dia_sem)))

head(NacionalDat)

# Obtenemos los dias sabados y domingos y reemplazamos sus valores por NA
NacionalDat$infectados[(NacionalDat$dia_sem %% 4) == 0 ] <- NA
NacionalDat$infectados[(NacionalDat$dia_sem %% 5) == 0 ] <- NA


week <-NacionalDat[complete.cases(NacionalDat),]

week

Warning message in cbind(consecutivo, dia_sem):
"number of rows of result is not a multiple of vector length (arg 2)"


,consecutivo,cve_ent,poblacion,nombre,dias,dia,mes,año,infectados,mest_txt,dia_sem,dias_txt
,<int>,<int>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<int>,<fct>,<int>,<fct>
1,1,0,127792286,Nacional,2020-02-26,26,02,2020,0,Feb,1,Miercoles
2,2,0,127792286,Nacional,2020-02-27,27,02,2020,4,Feb,2,Jueves
3,3,0,127792286,Nacional,2020-02-28,28,02,2020,2,Feb,3,Viernes
4,4,0,127792286,Nacional,2020-02-29,29,02,2020,2,Feb,4,Sabado
5,5,0,127792286,Nacional,2020-03-01,01,03,2020,2,Mar,5,Domingo
6,6,0,127792286,Nacional,2020-03-02,02,03,2020,5,Mar,6,Lunes


,consecutivo,cve_ent,poblacion,nombre,dias,dia,mes,año,infectados,mest_txt,dia_sem,dias_txt
,<int>,<int>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<int>,<fct>,<int>,<fct>
1,1,0,127792286,Nacional,2020-02-26,26,02,2020,0,Feb,1,Miercoles
2,2,0,127792286,Nacional,2020-02-27,27,02,2020,4,Feb,2,Jueves
3,3,0,127792286,Nacional,2020-02-28,28,02,2020,2,Feb,3,Viernes
6,6,0,127792286,Nacional,2020-03-02,02,03,2020,5,Mar,6,Lunes
7,7,0,127792286,Nacional,2020-03-03,03,03,2020,9,Mar,7,Martes
8,8,0,127792286,Nacional,2020-03-04,04,03,2020,10,Mar,1,Miercoles
9,9,0,127792286,Nacional,2020-03-05,05,03,2020,11,Mar,2,Jueves
10,10,0,127792286,Nacional,2020-03-06,06,03,2020,6,Mar,3,Viernes
13,13,0,127792286,Nacional,2020-03-09,09,03,2020,5,Mar,6,Lunes


In [23]:
# Otra forma de hacerlo
datos$diasem<-format(as.Date(datos$dias),"%A")



datos$inf<-ifelse(datos$diasem%in%c("sábado","domingo"),
              NA,datos$infectados)


filter(datos, is.na(inf))


cve_ent,poblacion,nombre,dias,dia,mes,año,infectados,mest_txt,consecutivo,diasem,inf
<int>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<int>,<fct>,<int>,<chr>,<int>
1,1434635,AGUASCALIENTES,2020-02-29,29,02,2020,0,Feb,100,sábado,NA
2,3634868,BAJA CALIFORNIA,2020-02-29,29,02,2020,0,Feb,101,sábado,NA
3,804708,BAJA CALIFORNIA SUR,2020-02-29,29,02,2020,0,Feb,102,sábado,NA
4,1000617,CAMPECHE,2020-02-29,29,02,2020,0,Feb,103,sábado,NA
7,5730367,CHIAPAS,2020-02-29,29,02,2020,1,Feb,104,sábado,NA
8,3801487,CHIHUAHUA,2020-02-29,29,02,2020,0,Feb,105,sábado,NA
9,9018645,DISTRITO FEDERAL,2020-02-29,29,02,2020,0,Feb,106,sábado,NA
5,3218720,COAHUILA,2020-02-29,29,02,2020,0,Feb,107,sábado,NA
6,785153,COLIMA,2020-02-29,29,02,2020,0,Feb,108,sábado,NA
